In [1]:
# --- matplotlib --------------------------------------------------------------
import matplotlib.pyplot as plt

# --- Our own modules ---------------------------------------------------------
import sys

# permatent location of functions from pyFitXPS
sys.path.insert(0, '/home/julio/Python/pyFitXPS/functions/') # Julio Laptop
#sys.path.insert(0, '/Users/ceciliapatrault/Desktop/Julio/XPS_PYTHON/paquetes_funciones') # Mac
import specs_xy as sp

In [2]:
folder_path = "data/separated/"

In [3]:
archivos = sp.list_files_xy('data/')

data/


In [5]:
archivos

In [3]:
Au_MeMBA_5min = sp.load_all_file_xy_in(folder_path)

In [4]:
Au_MeMBA_5min.keys()

dict_keys(['VB (3)', 'S (3)', 'C (1)', 'O (2)', 'VB (0)', 'O_3', 'C (0)', 'VB (1)', 'VB (2)', 'C (3)', 'C (2)', 'S (0)', 'O (0)', 'Au (0)', 'Au (1)', 'Au (3)', 'O (1)', 'Survey (0)', 'S (2)', 'S (1)', 'Au (2)'])

In [5]:
Au_MeMBA_5min['VB (3)'].keys()

dict_keys(['details', 'data_orig'])

In [6]:
files = sorted(os.listdir(folder_path))

In [7]:
files

['Au_MeMBA_5min_10_Survey (0).xy',
 'Au_MeMBA_5min_11_S (0).xy',
 'Au_MeMBA_5min_12_C (0).xy',
 'Au_MeMBA_5min_13_Au (0).xy',
 'Au_MeMBA_5min_14_VB (0).xy',
 'Au_MeMBA_5min_15_O (0).xy',
 'Au_MeMBA_5min_16_S (1).xy',
 'Au_MeMBA_5min_17_C (1).xy',
 'Au_MeMBA_5min_18_Au (1).xy',
 'Au_MeMBA_5min_19_VB (1).xy',
 'Au_MeMBA_5min_20_O (1).xy',
 'Au_MeMBA_5min_21_S (2).xy',
 'Au_MeMBA_5min_22_C (2).xy',
 'Au_MeMBA_5min_23_Au (2).xy',
 'Au_MeMBA_5min_24_VB (2).xy',
 'Au_MeMBA_5min_25_O (2).xy',
 'Au_MeMBA_5min_26_S (3).xy',
 'Au_MeMBA_5min_27_C (3).xy',
 'Au_MeMBA_5min_28_Au (3).xy',
 'Au_MeMBA_5min_29_VB (3).xy',
 'Au_MeMBA_5min_30_O (3).xy']

In [4]:
for filename in files:
    print(folder_path + filename)

data/separated/Au_MeMBA_5min_10_Survey (0).xy
data/separated/Au_MeMBA_5min_11_S (0).xy
data/separated/Au_MeMBA_5min_12_C (0).xy
data/separated/Au_MeMBA_5min_13_Au (0).xy
data/separated/Au_MeMBA_5min_14_VB (0).xy
data/separated/Au_MeMBA_5min_15_O (0).xy
data/separated/Au_MeMBA_5min_16_S (1).xy
data/separated/Au_MeMBA_5min_17_C (1).xy
data/separated/Au_MeMBA_5min_18_Au (1).xy
data/separated/Au_MeMBA_5min_19_VB (1).xy
data/separated/Au_MeMBA_5min_20_O (1).xy
data/separated/Au_MeMBA_5min_21_S (2).xy
data/separated/Au_MeMBA_5min_22_C (2).xy
data/separated/Au_MeMBA_5min_23_Au (2).xy
data/separated/Au_MeMBA_5min_24_VB (2).xy
data/separated/Au_MeMBA_5min_25_O (2).xy
data/separated/Au_MeMBA_5min_26_S (3).xy
data/separated/Au_MeMBA_5min_27_C (3).xy
data/separated/Au_MeMBA_5min_28_Au (3).xy
data/separated/Au_MeMBA_5min_29_VB (3).xy
data/separated/Au_MeMBA_5min_30_O (3).xy
